In [1]:
import napari
from skimage.data import astronaut
from napari_picasso.napari_picasso import return_widget


viewer = napari.Viewer()
#widget = PicassoWidget(viewer)
widget = return_widget(viewer)
viewer.window.add_dock_widget(widget)
viewer.add_image(astronaut(), channel_axis=-1)

[<Image layer 'Image' at 0x210add8a7c8>,
 <Image layer 'Image [1]' at 0x210ade0a788>,
 <Image layer 'Image [2]' at 0x210ae095708>]

Using cpu
It 10 - total loss: 0.8261041641235352, total MI loss: 0.8109140992164612, total contrast loss: 0.015190064907073975
It 20 - total loss: 0.8247361183166504, total MI loss: 0.809617817401886, total contrast loss: 0.015118283219635487
It 30 - total loss: 0.8234671354293823, total MI loss: 0.8084245324134827, total contrast loss: 0.01504258532077074
It 40 - total loss: 0.8222651481628418, total MI loss: 0.8073008060455322, total contrast loss: 0.01496434211730957
It 50 - total loss: 0.8210311532020569, total MI loss: 0.8061438798904419, total contrast loss: 0.014887279830873013
It 60 - total loss: 0.8194373250007629, total MI loss: 0.8046280741691589, total contrast loss: 0.014809233136475086
It 70 - total loss: 0.8174694180488586, total MI loss: 0.8027408719062805, total contrast loss: 0.014728546142578125
It 80 - total loss: 0.8151698112487793, total MI loss: 0.8005246520042419, total contrast loss: 0.014645141549408436
It 90 - total loss: 0.8122284412384033, total MI loss: 0.

In [2]:
from napari.utils import progress
from time import sleep
from napari.qt.threading import thread_worker


pbr = progress(range(10))
def pbr_update(i):
    pbr.update(i)
    if i == 9:
        pbr.close()
    print(f'from yield_value {i}')
    
@thread_worker(connect={"yielded": pbr_update})
def long_running(steps=10, repeats=4, delay=1):
    for i in pbr:
        print(i)
        yield(i)
        sleep(delay)


    
worker = long_running()


# steps = 10
# pbr = progress(range(steps))
# for i in long_running(steps):
#     print(i)
#     pbr.update(i)
# pbr.close()
    


0
from yield_value 0
1
from yield_value 1
2
from yield_value 2
3
from yield_value 3
4
from yield_value 4
5
from yield_value 5
6
from yield_value 6
7
from yield_value 7
8
from yield_value 8
9
from yield_value 9
Using cpu
It 10 - total loss: 0.8252242207527161, total MI loss: 0.8099648952484131, total contrast loss: 0.01525931991636753
It 20 - total loss: 0.8239518404006958, total MI loss: 0.8086955547332764, total contrast loss: 0.015256292186677456
It 30 - total loss: 0.822663426399231, total MI loss: 0.8074032068252563, total contrast loss: 0.015260196290910244
It 40 - total loss: 0.8212747573852539, total MI loss: 0.8060032725334167, total contrast loss: 0.01527147926390171
It 50 - total loss: 0.8192973732948303, total MI loss: 0.8040131330490112, total contrast loss: 0.015284234657883644
It 60 - total loss: 0.8171197175979614, total MI loss: 0.8018242716789246, total contrast loss: 0.0152954226359725
It 70 - total loss: 0.8144541382789612, total MI loss: 0.799152135848999, total con

In [5]:
from napari.utils import progress
pbr = progress(range(5,10))
pbr.total

5

In [10]:
widget.run_btn.bind(True)
widget.run_btn.bind(False)
widget.run_btn.get_value()

False

Using cpu
It 10 - total loss: 1.79609215259552, total MI loss: 1.6190526485443115, total contrast loss: 0.1770394742488861
It 20 - total loss: 1.793644666671753, total MI loss: 1.6173226833343506, total contrast loss: 0.17632201313972473
It 30 - total loss: 1.7902164459228516, total MI loss: 1.6147146224975586, total contrast loss: 0.1755017638206482
It 40 - total loss: 1.7855160236358643, total MI loss: 1.6109461784362793, total contrast loss: 0.17456978559494019
It 50 - total loss: 1.7789340019226074, total MI loss: 1.6054050922393799, total contrast loss: 0.17352886497974396
It 60 - total loss: 1.770835041999817, total MI loss: 1.5984455347061157, total contrast loss: 0.17238953709602356
It 70 - total loss: 1.7589645385742188, total MI loss: 1.5878045558929443, total contrast loss: 0.17115992307662964
It 80 - total loss: 1.7408010959625244, total MI loss: 1.570955514907837, total contrast loss: 0.16984562575817108
It 90 - total loss: 1.7149162292480469, total MI loss: 1.546468615531

In [4]:
widget['run_btn']

AttributeError: 'PicassoWidget' object has no attribute 'run_btn'

In [6]:
test = widget.train_model()
test.start()

training
in for loop
It 10 - total loss: 1.7959944009780884, total MI loss: 1.6189546585083008, total contrast loss: 0.1770397126674652
It 20 - total loss: 1.7938904762268066, total MI loss: 1.617567539215088, total contrast loss: 0.17632290720939636
It 30 - total loss: 1.79129159450531, total MI loss: 1.6157886981964111, total contrast loss: 0.17550288140773773
It 40 - total loss: 1.787611722946167, total MI loss: 1.6130425930023193, total contrast loss: 0.17456918954849243
It 50 - total loss: 1.7816733121871948, total MI loss: 1.6081480979919434, total contrast loss: 0.17352516949176788
It 60 - total loss: 1.7726953029632568, total MI loss: 1.6003142595291138, total contrast loss: 0.17238101363182068
It 70 - total loss: 1.7594220638275146, total MI loss: 1.5882782936096191, total contrast loss: 0.1711438000202179
It 80 - total loss: 1.739220380783081, total MI loss: 1.5694035291671753, total contrast loss: 0.16981683671474457
It 90 - total loss: 1.709040641784668, total MI loss: 1.54

In [7]:
def out_test():
    i = 1
    def in_test():
        print(i)
    in_test()
        
        
out_test()

1


In [15]:
np.where(mm[0,:,i] == 1)

(array([1], dtype=int64),)

In [14]:
np.where(mm[0,:,i] == 1)[0][0]

1

In [21]:
viewer.layers[0].as_layer_data_tuple()[2]

'image'

In [11]:
isinstance(viewer.layers[0], 'napari.layers.image.image.Image')

TypeError: isinstance() arg 2 must be a type or tuple of types

In [18]:
import numpy as np
napari.layers.Layer(np.zeros((10,10)))

TypeError: Can't instantiate abstract class Layer with abstract methods _extent_data, _get_ndim, _get_state, _get_value, _set_view_slice, _update_thumbnail, data